In [8]:
# Script Name: Option_1 (Arbitrage Option1 Sequential - Real Trade Execution, Delta Neutral)
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo

# API Keys
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
BITGET_API_PASSWORD = 'BgSecure789Trade2025'

# Connect to APIs
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

In [10]:
# Settings
SPREAD_THRESHOLD = 0.4 / 100
MIN_VOLUME = 50000
CHECK_INTERVAL = 2  # Faster scanner
TRADE_USD_SIZE = 10
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")
TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
exchanges = {'Bybit': bybit, 'Bitget': bitget}
futures_markets_cache = {}

# Helper Functions
def ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
    timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%H:%M:%S')
    prompt = (f"\n⏰ {timestamp} | ⚠️ Arbitrage Signal on {exchange_name} - {symbol}\n"
              f"Spread: {spread:.4%} | Net Profit: {net_profit:.4%}\n"
              f"Do you want to execute this trade for {exchange_name} {symbol}? (yes/no): ")
    choice = input(prompt).strip().lower()
    return choice == 'yes'

def get_best_bid_ask(exchange, symbol):
    order_book = exchange.fetch_order_book(symbol)
    best_bid = order_book['bids'][0][0] if order_book['bids'] else None
    best_ask = order_book['asks'][0][0] if order_book['asks'] else None
    return best_bid, best_ask

def execute_trade(exchange_name, symbol, spot_price, futures_price):
    print(f"🚀 Executing REAL trade on {exchange_name} - {symbol}")
    try:
        exchange = exchanges[exchange_name]
        base_asset = symbol.split('/')[0]
        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)

        amount = TRADE_USD_SIZE / spot_price
        best_bid, _ = get_best_bid_ask(exchange, symbol)
        if not best_bid:
            print("❌ Cannot determine best bid.")
            return

        print(f"📈 Placing SPOT LIMIT BUY: {amount:.6f} {base_asset} @ {best_bid:.4f}")
        order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side='buy',
            amount=amount,
            price=best_bid
        )

        max_wait = 15
        wait_time = 0
        filled = 0
        while wait_time < max_wait:
            current_order = exchange.fetch_order(order['id'], symbol, params={"acknowledged": True})
            filled = current_order['filled']
            remaining = current_order['remaining']
            current_price = get_best_bid_ask(exchange, symbol)[0]

            if filled > 0:
                break

            if current_order['price'] != current_price:
                print(f"🔁 Adjusting spot price to new best bid {current_price:.4f}")
                exchange.cancel_order(order['id'], symbol)
                order = exchange.create_order(
                    symbol=symbol,
                    type='limit',
                    side='buy',
                    amount=amount,
                    price=current_price
                )
            time.sleep(1)
            wait_time += 1

        if filled == 0:
            print("❌ Spot order did not fill. Aborting futures leg.")
            exchange.cancel_order(order['id'], symbol)
            return

        print(f"✅ Spot filled: {filled:.6f} {base_asset}. Proceeding with futures SELL.")

        if exchange_name == "Bybit":
            futures_exchange = ccxt.bybit({
                'apiKey': BYBIT_API_KEY,
                'secret': BYBIT_API_SECRET,
                'enableRateLimit': True,
                'options': {'defaultType': 'future'}
            })
        elif exchange_name == "Bitget":
            futures_exchange = ccxt.bitget({
                'apiKey': BITGET_API_KEY,
                'secret': BITGET_API_SECRET,
                'password': BITGET_API_PASSWORD,
                'enableRateLimit': True,
                'options': {'defaultType': 'swap'}
            })
        else:
            print(f"❌ Unsupported exchange for futures leg: {exchange_name}")
            return

        futures_exchange.set_leverage(1, futures_symbol)
        order_futures = futures_exchange.create_order(
            symbol=futures_symbol,
            type='market',
            side='sell',
            amount=filled
        )

        print(f"✅ Futures Order Placed: {order_futures['id']}")
        print(f"🏁 Trade Complete for {symbol} | Amount: {filled:.6f}")
    except Exception as e:
        print(f"❌ Trade Error: {e}")

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [symbol for symbol in markets if ":USDT" in symbol or "-USDT-SWAP" in symbol]
        for symbol, market in markets.items():
            if exchange_name == "Bitget" and not market.get('spot', False):
                continue
            if '/USDT' in symbol and ':USDT' not in symbol and '-USDT-SWAP' not in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last", None)
    except Exception:
        return None

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT")]
    return match[0] if match else None

def check_intra_exchange_arbitrage(exchange, exchange_name, symbol, spot_price, futures_price):
    trading_fee = TRADING_FEES.get(exchange_name, 0.001)
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (trading_fee * 2)
    if net_profit >= SPREAD_THRESHOLD:
        timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S %Z')
        print(f"\n🔍 Arbitrage Opportunity on {exchange_name}: {symbol} | {timestamp}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        if ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
            execute_trade(exchange_name, symbol, spot_price, futures_price)

def process_tokens(exchange, exchange_name, tokens):
    futures_list = futures_markets_cache.get(exchange_name, [])
    for token in tokens:
        futures_symbol = find_futures_symbol(token, futures_list)
        if not futures_symbol:
            continue
        spot_price = get_price(exchange, token)
        futures_price = get_price(exchange, futures_symbol)
        if spot_price and futures_price:
            check_intra_exchange_arbitrage(exchange, exchange_name, token, spot_price, futures_price)

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tradable_tokens = get_tradable_tokens(exchange, exchange_name)
                if not tradable_tokens:
                    print(f"❌ No tradable tokens found on {exchange_name}. Retrying in 10 seconds...")
                    time.sleep(10)
                    continue
                print(f"🔄 Checking for arbitrage opportunities on {exchange_name}...")
                process_tokens(exchange, exchange_name, tradable_tokens)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()

✅ Found 397 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 550 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...
✅ Cycle complete. Waiting for next check...
✅ Found 396 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 550 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...

🔍 Arbitrage Opportunity on Bitget: GRIFFAIN/USDT | 2025-04-19 00:39:59 WIB
    ✅ Spot Price: 0.02790 USDT
    ✅ Futures Price: 0.02806 USDT
    ✅ Spread: 0.5735%
    ✅ Net Profit After Fees: 0.4135%



⏰ 00:39:59 | ⚠️ Arbitrage Signal on Bitget - GRIFFAIN/USDT
Spread: 0.5735% | Net Profit: 0.4135%
Do you want to execute this trade for Bitget GRIFFAIN/USDT? (yes/no):  yes


🚀 Executing REAL trade on Bitget - GRIFFAIN/USDT
📈 Placing SPOT LIMIT BUY: 358.422939 GRIFFAIN @ 0.0279
❌ Spot order did not fill. Aborting futures leg.
✅ Cycle complete. Waiting for next check...
✅ Found 396 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 550 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...

🔍 Arbitrage Opportunity on Bitget: KOMA/USDT | 2025-04-19 00:52:16 WIB
    ✅ Spot Price: 0.01760 USDT
    ✅ Futures Price: 0.01772 USDT
    ✅ Spread: 0.6818%
    ✅ Net Profit After Fees: 0.5218%



⏰ 00:52:16 | ⚠️ Arbitrage Signal on Bitget - KOMA/USDT
Spread: 0.6818% | Net Profit: 0.5218%
Do you want to execute this trade for Bitget KOMA/USDT? (yes/no):  yes


🚀 Executing REAL trade on Bitget - KOMA/USDT
📈 Placing SPOT LIMIT BUY: 568.181818 KOMA @ 0.0176
❌ Spot order did not fill. Aborting futures leg.


KeyboardInterrupt: 

In [12]:
# Script Name: Option_1 (Arbitrage Option1 Sequential - Delta Neutral Real Trade)
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo

# API Keys
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
BITGET_API_PASSWORD = 'BgSecure789Trade2025'

# Connect to APIs
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

In [14]:
# Settings
SPREAD_THRESHOLD = 0.4 / 100
MIN_VOLUME = 50000
CHECK_INTERVAL = 5
TRADE_USD_SIZE = 10
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")
TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
exchanges = {'Bybit': bybit, 'Bitget': bitget}
futures_markets_cache = {}

# Helper Functions
def ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
    timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%H:%M:%S')
    prompt = (f"\n⏰ {timestamp} | ⚠️ Arbitrage Signal on {exchange_name} - {symbol}\n"
              f"Spread: {spread:.4%} | Net Profit: {net_profit:.4%}\n"
              f"Do you want to execute this trade for {exchange_name} {symbol}? (yes/no): ")
    choice = input(prompt).strip().lower()
    return choice == 'yes'

def wait_for_fill(exchange, order_id, symbol):
    print("🕒 Waiting for spot order to fill...")
    while True:
        order = exchange.fetch_order(order_id, symbol)
        if order['status'] == 'closed':
            print("✅ Spot order filled.")
            return order['filled']
        print("⏳ Still waiting for fill...")
        time.sleep(2)

def execute_trade(exchange_name, symbol, spot_price, futures_price):
    print(f"🚀 Executing REAL delta-neutral trade on {exchange_name} - {symbol}")
    try:
        exchange = exchanges[exchange_name]
        base_asset = symbol.split('/')[0]
        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)

        # Calculate quantity
        amount = TRADE_USD_SIZE / spot_price

        # Spot BUY (limit order)
        print(f"📈 Placing SPOT BUY: {amount:.6f} {base_asset} @ {spot_price:.4f}")
        order_spot = exchange.create_order(
            symbol=symbol,
            type='limit',
            side='buy',
            amount=amount,
            price=spot_price
        )

        filled_amount = wait_for_fill(exchange, order_spot['id'], symbol)
        if filled_amount == 0:
            return

        print(f"✅ Spot Order Filled: {filled_amount:.6f} {base_asset}")

        # Futures SELL (market order)
        print(f"📉 Placing FUTURES SELL: {filled_amount:.6f} {base_asset} @ market")
        futures_exchange = ccxt.bybit({
            'apiKey': BYBIT_API_KEY,
            'secret': BYBIT_API_SECRET,
            'enableRateLimit': True,
            'options': {'defaultType': 'future'}
        })

        futures_exchange.set_leverage(1, futures_symbol)
        order_futures = futures_exchange.create_order(
            symbol=futures_symbol,
            type='market',
            side='sell',
            amount=filled_amount
        )

        print(f"✅ Futures Order Placed: {order_futures['id']}")
        print(f"🏁 Delta-neutral trade completed for {symbol}")
    except Exception as e:
        print(f"❌ Trade Error: {e}")

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [symbol for symbol in markets if ":USDT" in symbol or "-USDT-SWAP" in symbol]
        for symbol, market in markets.items():
            if exchange_name == "Bitget" and not market.get('spot', False):
                continue
            if '/USDT' in symbol and ':USDT' not in symbol and '-USDT-SWAP' not in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last", None)
    except Exception:
        return None

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT")]
    return match[0] if match else None

def check_intra_exchange_arbitrage(exchange, exchange_name, symbol, spot_price, futures_price):
    trading_fee = TRADING_FEES.get(exchange_name, 0.001)
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (trading_fee * 2)
    if net_profit >= SPREAD_THRESHOLD:
        timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S %Z')
        print(f"\n🔍 Arbitrage Opportunity on {exchange_name}: {symbol} | {timestamp}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        if ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
            execute_trade(exchange_name, symbol, spot_price, futures_price)

def process_tokens(exchange, exchange_name, tokens):
    futures_list = futures_markets_cache.get(exchange_name, [])
    for token in tokens:
        futures_symbol = find_futures_symbol(token, futures_list)
        if not futures_symbol:
            continue
        spot_price = get_price(exchange, token)
        futures_price = get_price(exchange, futures_symbol)
        if spot_price and futures_price:
            check_intra_exchange_arbitrage(exchange, exchange_name, token, spot_price, futures_price)

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tradable_tokens = get_tradable_tokens(exchange, exchange_name)
                if not tradable_tokens:
                    print(f"❌ No tradable tokens found on {exchange_name}. Retrying in 10 seconds...")
                    time.sleep(10)
                    continue
                print(f"🔄 Checking for arbitrage opportunities on {exchange_name}...")
                process_tokens(exchange, exchange_name, tradable_tokens)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()


✅ Found 395 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 547 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...

🔍 Arbitrage Opportunity on Bitget: GRIFFAIN/USDT | 2025-04-19 01:11:05 WIB
    ✅ Spot Price: 0.02750 USDT
    ✅ Futures Price: 0.02767 USDT
    ✅ Spread: 0.6182%
    ✅ Net Profit After Fees: 0.4582%



⏰ 01:11:05 | ⚠️ Arbitrage Signal on Bitget - GRIFFAIN/USDT
Spread: 0.6182% | Net Profit: 0.4582%
Do you want to execute this trade for Bitget GRIFFAIN/USDT? (yes/no):  yes


🚀 Executing REAL delta-neutral trade on Bitget - GRIFFAIN/USDT
📈 Placing SPOT BUY: 363.636364 GRIFFAIN @ 0.0275
🕒 Waiting for spot order to fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still wait

KeyboardInterrupt: Interrupted by user

#udah berjalan tapi beda exchange. Bagian atas ya

In [13]:
# Script Name: Option_1 (Arbitrage Option1 Sequential - Delta Neutral Real Trade)
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo

# API Keys
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
BITGET_API_PASSWORD = 'BgSecure789Trade2025'

# Connect to APIs
bybit_spot = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

bitget_spot = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

bybit_futures = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'defaultType': 'future'}
})

bitget_futures = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'defaultType': 'swap'}
})


In [15]:
# Settings
SPREAD_THRESHOLD = 0.4 / 100
MIN_VOLUME = 500000
CHECK_INTERVAL = 2
TRADE_USD_SIZE = 10
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")
TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
exchanges = {'Bybit': bybit_spot, 'Bitget': bitget_spot}
futures_apis = {'Bybit': bybit_futures, 'Bitget': bitget_futures}
futures_markets_cache = {}

# Helper Functions
def ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
    timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%H:%M:%S')
    prompt = (f"\n⏰ {timestamp} | ⚠️ Arbitrage Signal on {exchange_name} - {symbol}\n"
              f"Spread: {spread:.4%} | Net Profit: {net_profit:.4%}\n"
              f"Do you want to execute this trade for {exchange_name} {symbol}? (yes/no): ")
    choice = input(prompt).strip().lower()
    return choice == 'yes'

def wait_for_fill(exchange, order_id, symbol):
    print("🕒 Waiting for spot order to fill...")
    while True:
        order = exchange.fetch_order(order_id, symbol)
        if order['status'] == 'closed':
            print("✅ Spot order filled.")
            return order['filled']
        print("⏳ Still waiting for fill...")
        time.sleep(2)

def execute_trade(exchange_name, symbol, spot_price, futures_price):
    print(f"🚀 Executing REAL delta-neutral trade on {exchange_name} - {symbol}")
    try:
        spot_exchange = exchanges[exchange_name]
        futures_exchange = futures_apis[exchange_name]
        base_asset = symbol.split('/')[0]
        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)

        # Calculate quantity
        amount = TRADE_USD_SIZE / spot_price

        # Spot BUY (limit order)
        print(f"📈 Placing SPOT LIMIT BUY: {amount:.6f} {base_asset} @ {spot_price:.4f}")
        order_spot = spot_exchange.create_order(
            symbol=symbol,
            type='limit',
            side='buy',
            amount=amount,
            price=spot_price
        )

        filled_amount = wait_for_fill(spot_exchange, order_spot['id'], symbol)
        if filled_amount == 0:
            return

        print(f"✅ Spot Order Filled: {filled_amount:.6f} {base_asset}")

        # Futures SELL (market order)
        print(f"📉 Placing FUTURES MARKET SELL: {filled_amount:.6f} {base_asset}")
        if exchange_name == 'Bitget':
            futures_exchange.set_position_mode(False, futures_symbol)
        futures_exchange.set_leverage(1, futures_symbol)
        order_futures = futures_exchange.create_order(
            symbol=futures_symbol,
            type='market',
            side='sell',
            amount=filled_amount
        )

        print(f"✅ Futures Order Placed: {order_futures['id']}")
        print(f"🏁 Delta-neutral trade completed for {symbol} on {exchange_name}")
    except Exception as e:
        print(f"❌ Trade Error: {e}")

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [symbol for symbol in markets if ":USDT" in symbol or "-USDT-SWAP" in symbol or ":USDT:USDT" in symbol]
        for symbol, market in markets.items():
            if exchange_name == "Bitget" and not market.get('spot', False):
                continue
            if '/USDT' in symbol and ':USDT' not in symbol and '-USDT-SWAP' not in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last", None)
    except Exception:
        return None

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT") or f.startswith(f"{base}_USDT")]
    return match[0] if match else None

def check_intra_exchange_arbitrage(exchange, exchange_name, symbol, spot_price, futures_price):
    trading_fee = TRADING_FEES.get(exchange_name, 0.001)
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (trading_fee * 2)
    if net_profit >= SPREAD_THRESHOLD:
        timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S %Z')
        print(f"\n🔍 Arbitrage Opportunity on {exchange_name}: {symbol} | {timestamp}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        if ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
            execute_trade(exchange_name, symbol, spot_price, futures_price)

def process_tokens(exchange, exchange_name, tokens):
    futures_list = futures_markets_cache.get(exchange_name, [])
    for token in tokens:
        futures_symbol = find_futures_symbol(token, futures_list)
        if not futures_symbol:
            continue
        spot_price = get_price(exchange, token)
        futures_price = get_price(futures_apis[exchange_name], futures_symbol)
        if spot_price and futures_price:
            check_intra_exchange_arbitrage(exchange, exchange_name, token, spot_price, futures_price)

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tradable_tokens = get_tradable_tokens(exchange, exchange_name)
                if not tradable_tokens:
                    print(f"❌ No tradable tokens found on {exchange_name}. Retrying in 10 seconds...")
                    time.sleep(10)
                    continue
                print(f"🔄 Checking for arbitrage opportunities on {exchange_name}...")
                process_tokens(exchange, exchange_name, tradable_tokens)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()


✅ Found 208 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 230 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...

🔍 Arbitrage Opportunity on Bitget: KOMA/USDT | 2025-04-20 23:38:05 WIB
    ✅ Spot Price: 0.01770 USDT
    ✅ Futures Price: 0.01784 USDT
    ✅ Spread: 0.7910%
    ✅ Net Profit After Fees: 0.6310%



⏰ 23:38:05 | ⚠️ Arbitrage Signal on Bitget - KOMA/USDT
Spread: 0.7910% | Net Profit: 0.6310%
Do you want to execute this trade for Bitget KOMA/USDT? (yes/no):  yes


🚀 Executing REAL delta-neutral trade on Bitget - KOMA/USDT
📈 Placing SPOT LIMIT BUY: 564.971751 KOMA @ 0.0177
🕒 Waiting for spot order to fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waiting for fill...
⏳ Still waitin


⏰ 23:52:14 | ⚠️ Arbitrage Signal on Bitget - BGSC/USDT
Spread: 0.5675% | Net Profit: 0.4075%
Do you want to execute this trade for Bitget BGSC/USDT? (yes/no):  no


✅ Cycle complete. Waiting for next check...
✅ Found 208 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 230 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...

🔍 Arbitrage Opportunity on Bitget: ARC/USDT | 2025-04-20 23:58:21 WIB
    ✅ Spot Price: 0.03540 USDT
    ✅ Futures Price: 0.03560 USDT
    ✅ Spread: 0.5650%
    ✅ Net Profit After Fees: 0.4050%



⏰ 23:58:21 | ⚠️ Arbitrage Signal on Bitget - ARC/USDT
Spread: 0.5650% | Net Profit: 0.4050%
Do you want to execute this trade for Bitget ARC/USDT? (yes/no):  no


✅ Cycle complete. Waiting for next check...
✅ Found 208 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 231 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...

🔍 Arbitrage Opportunity on Bitget: KILO/USDT | 2025-04-21 00:02:10 WIB
    ✅ Spot Price: 0.03772 USDT
    ✅ Futures Price: 0.03800 USDT
    ✅ Spread: 0.7450%
    ✅ Net Profit After Fees: 0.5850%



⏰ 00:02:10 | ⚠️ Arbitrage Signal on Bitget - KILO/USDT
Spread: 0.7450% | Net Profit: 0.5850%
Do you want to execute this trade for Bitget KILO/USDT? (yes/no):  no


✅ Cycle complete. Waiting for next check...
✅ Found 208 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 232 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...
✅ Cycle complete. Waiting for next check...
✅ Found 207 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 232 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...


E:\Users\bb_bu\anaconda3\Lib\collections\__init__.py:449: RuntimeWarning: coroutine 'InteractiveShell.run_cell_async' was never awaited
  result = tuple_new(cls, iterable)

KeyboardInterrupt



# IT WORK FINALLY DONT CHANGE THIS CODE

Code di Bawah:
✅ When you say "yes" to a trade, spot order is placed immediately.

✅ Then continue scanning for other arbitrage opportunities without waiting.

✅ Meanwhile, background process keeps checking if previous spot orders are filled.

✅ When a spot order is filled, automatically open the futures hedge, and notify you.

In [ ]:
# Script Name: Option_1 (Arbitrage Option1 Sequential - Delta Neutral Real Trade - Async Version)
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo
import threading

# API Keys
BYBIT_API_KEY = '----'
BYBIT_API_SECRET = '-------'
BITGET_API_KEY = '------'
BITGET_API_SECRET = '------'
BITGET_API_PASSWORD = '------'

# Connect to APIs
bybit_spot = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

bitget_spot = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

bybit_futures = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'defaultType': 'future'}
})

bitget_futures = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'defaultType': 'swap'}
})

# Settings
SPREAD_THRESHOLD = 0.4 / 100
MIN_VOLUME = 500000
CHECK_INTERVAL = 2
TRADE_USD_SIZE = 10
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")
TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
exchanges = {'Bybit': bybit_spot, 'Bitget': bitget_spot}
futures_apis = {'Bybit': bybit_futures, 'Bitget': bitget_futures}
futures_markets_cache = {}
open_spot_orders = []

# Helper Functions
def ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
    timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%H:%M:%S')
    prompt = (f"\n⏰ {timestamp} | ⚠️ Arbitrage Signal on {exchange_name} - {symbol}\n"
              f"Spread: {spread:.4%} | Net Profit: {net_profit:.4%}\n"
              f"Do you want to execute this trade for {exchange_name} {symbol}? (yes/no): ")
    choice = input(prompt).strip().lower()
    return choice == 'yes'

def monitor_fills():
    while True:
        for order_info in open_spot_orders[:]:
            spot_exchange = order_info['spot_exchange']
            futures_exchange = order_info['futures_exchange']
            futures_symbol = order_info['futures_symbol']
            order_id = order_info['order_id']
            symbol = order_info['symbol']
            exchange_name = order_info['exchange_name']
            base_asset = order_info['base_asset']
            try:
                order = spot_exchange.fetch_order(order_id, symbol)
                if order['status'] == 'closed':
                    filled_amount = order['filled']
                    print(f"✅ Spot order filled for {symbol} on {exchange_name}!")
                    print(f"📉 Placing Futures SHORT: {filled_amount:.6f} {base_asset}")
                    if exchange_name == 'Bitget':
                        futures_exchange.set_position_mode(False, futures_symbol)
                    futures_exchange.set_leverage(1, futures_symbol)
                    order_futures = futures_exchange.create_order(
                        symbol=futures_symbol,
                        type='market',
                        side='sell',
                        amount=filled_amount
                    )
                    print(f"✅ Futures order placed: {order_futures['id']}")
                    open_spot_orders.remove(order_info)
            except Exception as e:
                print(f"❌ Error monitoring order: {e}")
        time.sleep(2)

def execute_trade(exchange_name, symbol, spot_price, futures_price):
    print(f"🚀 Initiating delta-neutral trade on {exchange_name} - {symbol}")
    try:
        spot_exchange = exchanges[exchange_name]
        futures_exchange = futures_apis[exchange_name]
        base_asset = symbol.split('/')[0]
        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)

        amount = TRADE_USD_SIZE / spot_price

        print(f"📈 Placing SPOT LIMIT BUY: {amount:.6f} {base_asset} @ {spot_price:.4f}")
        order_spot = spot_exchange.create_order(
            symbol=symbol,
            type='limit',
            side='buy',
            amount=amount,
            price=spot_price
        )

        open_spot_orders.append({
            'spot_exchange': spot_exchange,
            'futures_exchange': futures_exchange,
            'futures_symbol': futures_symbol,
            'order_id': order_spot['id'],
            'symbol': symbol,
            'exchange_name': exchange_name,
            'base_asset': base_asset
        })

        print(f"✅ Spot order placed: {order_spot['id']} (Now monitoring fill...)\n")
    except Exception as e:
        print(f"❌ Trade Error: {e}")

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [symbol for symbol in markets if ":USDT" in symbol or "-USDT-SWAP" in symbol or ":USDT:USDT" in symbol]
        for symbol, market in markets.items():
            if exchange_name == "Bitget" and not market.get('spot', False):
                continue
            if '/USDT' in symbol and ':USDT' not in symbol and '-USDT-SWAP' not in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last", None)
    except Exception:
        return None

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT") or f.startswith(f"{base}_USDT")]
    return match[0] if match else None

def check_intra_exchange_arbitrage(exchange, exchange_name, symbol, spot_price, futures_price):
    trading_fee = TRADING_FEES.get(exchange_name, 0.001)
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (trading_fee * 2)
    if net_profit >= SPREAD_THRESHOLD:
        timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S %Z')
        print(f"\n🔍 Arbitrage Opportunity on {exchange_name}: {symbol} | {timestamp}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        if ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
            execute_trade(exchange_name, symbol, spot_price, futures_price)

def process_tokens(exchange, exchange_name, tokens):
    futures_list = futures_markets_cache.get(exchange_name, [])
    for token in tokens:
        futures_symbol = find_futures_symbol(token, futures_list)
        if not futures_symbol:
            continue
        spot_price = get_price(exchange, token)
        futures_price = get_price(futures_apis[exchange_name], futures_symbol)
        if spot_price and futures_price:
            check_intra_exchange_arbitrage(exchange, exchange_name, token, spot_price, futures_price)

def main():
    threading.Thread(target=monitor_fills, daemon=True).start()
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tradable_tokens = get_tradable_tokens(exchange, exchange_name)
                if not tradable_tokens:
                    print(f"❌ No tradable tokens found on {exchange_name}. Retrying in 10 seconds...")
                    time.sleep(10)
                    continue
                print(f"🔄 Checking for arbitrage opportunities on {exchange_name}...")
                process_tokens(exchange, exchange_name, tradable_tokens)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")